In [ ]:
#Importing Modules
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import us
sns.set(style='whitegrid', color_codes=True) 
from IPython.display import display
import altair as alt
from vega_datasets import data
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv("cleaned_full_data.csv")
df.head()

## Calculate Swing

In [ ]:
p_gop_16 = df["p_dem_16"]
p_dem_16 = df["p_gop_16"]
del df["p_dem_16"]
del df["p_gop_16"]
df["p_dem_16"] = p_dem_16
df["p_gop_16"] = p_gop_16
df.head()

In [ ]:
df["c_dem"] = df["p_dem_20"] - df["p_dem_16"]
df["c_gop"] = df["p_gop_20"] - df["p_gop_16"]
df["swing_gop"] = (df["c_gop"]-df["c_dem"])/2
df["swing_dem"] = (df["c_dem"]-df["c_gop"])/2
df.head()

## Merging results/census data with Shape Data

In [ ]:
us_map = gpd.read_file("county_shape/cb_2019_us_county_500k.shp")
us_map.head()

In [ ]:
for i in us_map["AFFGEOID"]:
    if i not in list(df["geo_id"]):
        us_map = us_map[~us_map.AFFGEOID.str.contains(i)]
len(us_map)

In [ ]:
## Moving Hawaii
m = us_map.STATEFP == "15"
us_map[m] = us_map[m].set_geometry(us_map[m].translate(54))

In [ ]:
us_map["geo_id"] = us_map["AFFGEOID"]
del us_map["AFFGEOID"]
us_boundaries = pd.merge(us_map,df,on="geo_id")
len(us_boundaries)

In [ ]:
## State Data:
state_shape = gpd.read_file("state_shape/cb_2018_us_state_500k.shp")
len(state_shape)


In [ ]:
state_shape=state_shape[~state_shape.STATEFP.str.contains("02")]
len(state_shape)

In [ ]:
m = state_shape.STATEFP == "15"
state_shape[m] = state_shape[m].set_geometry(state_shape[m].translate(54))

In [ ]:
mapping = us.states.mapping("fips","name")
state = []
for i in state_shape["STATEFP"]:
    state.append(mapping[i])
state_shape["state_name"] = state

## Plot Swing Map

In [ ]:
data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

In [ ]:
state_geo = alt.InlineData(values = state_shape.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

In [ ]:
plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(
    color=alt.Color('properties.swing_dem:Q', scale=alt.Scale(scheme='bluered',domain=[0,1])),
    tooltip=['properties.county_name:N','properties.swing_dem:Q','properties.state_name:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)
alt.layer(plot,outline)

In [ ]:
plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(
    color=alt.Color('properties.p_gop_16:Q', scale=alt.Scale(scheme='bluered',domain=[0,1])),
    tooltip=['properties.county_name:N','properties.p_gop_16:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)
alt.layer(plot,outline)

In [ ]:
plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.p_gop_16:Q', scale=alt.Scale(scheme='bluered',domain=[0,1])),
    tooltip=['properties.county_name:N','properties.p_gop_16:Q','properties.state:N']
).properties(
    width=800,
    height=600
)

alt.layer(plot)

In [ ]:
plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.swing_dem:Q', scale=alt.Scale(scheme='redblue',domain=[-0.15,0.15])),
    tooltip=['properties.county_name:N','properties.swing_dem:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
alt.layer(plot)

To assess election results, two measures were used; swing from GOP to DEM and a baseline measure of democratic support in the 2016 election.
* Looking at the map, the distribution of these two metrics differ between the two maps, suggesting that factors driving each measure may be different and vary greatly between regions. 
* Outliers: States with deep red and blue bands, indicating that these states swung greater to the right and left respectively. 


In [ ]:
## Investigate the correlation between the two measures:
sns.scatterplot(df["p_gop_16"],df["swing"],hue=df["state"],marker="x")

In [ ]:
np.corrcoef(df["p_gop_16"],df["swing"])

* Calculating correlation also supports this hypothesis with a c = 0.13647053
* Distinct Regional Patterns are also observed, e.g. California strong support for democrats in 2016 election and swung further left, whereas Utah showed strong support for GOP and swung further right in the 2020 election.
* Also variation by county, e.g. Texas, counties who showed strong support for democrats in the 2016 election swung further right whereas counties which showed strong support for GOP had a slight swing towards the left.

In [ ]:
## Determine Features which have the same correlation with both measures

In [ ]:
df.columns

In [ ]:
features = ['total_pop', 'bachelors>', 'veterans',
       'white_collar', 'pink_collar', 'blue_collar', 'median_income',
       'no_health_insurance', 'p_male', 'p_white', 'p_black', 'p_asian',
       'p_latino', 'english_speaking', 'immigrants', 'gen_z', 'gen_y', 'gen_x',
       'baby_boomer', 'silent_gen']

type(features)

In [ ]:
fig,axes = plt.subplots(ncols=5, nrows=4,figsize=(30,20),sharey=True)
for i,f in zip(features,axes.flat):
    sns.regplot(df[i],df["p_gop_16"],ax=f,line_kws={"color": "red"})
plt.show()

In [ ]:
fig,axes = plt.subplots(ncols=5, nrows=4,figsize=(30,20),sharey=True)
for i,f in zip(features,axes.flat):
    sns.regplot(df[i],df["swing"],ax=f,line_kws={"color": "red"})
plt.show()

##### Positive Correlations:
1. total_pop (cities)
2. bachelors> (cities/education)
3. White collars (high Income)
4. Median Income (high Income)
5. p_asian (probably congregates in cities)
6. gen_z
7. gen_y
8. gen_x
 
##### Negative Correlations:
1. Blue Collar (poor)
2. No Health Insurance (poor)
3. immigrants (racism)
4. silent_gen (old)
5. p-White


## Modelling Voter Probability
Now we've found some variables that not only correlate with the share of Remain:Leave vote but also correspond to phenomena that might help explain the vote. We can try to build models to take account of the effects of these variables on the vote.

We will generate a multivariate regression model using the variables above that we hypothesise are discriminating. It's worth here quickly revisiting the assumptions of multivariate linear regression:

* Linear relationship between expected value of outcome and each explanatory variable
* No or limited collinearity of explanatory variables
* No (spatial) auto-correlation in residuals 
* Homoscedasticity (constant variance) in residuals
* Normality in distribution of residuals

We have already identified linearity in the relationships between our outcome and candidate explanatory variables and we'll discuss the distribution of model residuals shortly. However, we've yet to address the problem of collinearity of explanatory variables. Since we wish to develop a model for *explaining* voter preference, it's important that our model is parsimonious: that is, that we can explain the outcome with as few explanatory variables as possible. Attending to issues of collinearity helps us to do this: we can eliminate variables that effectively represent the same concept. 

Collinearity can initially be assessed through studying pairwise correlation between each explanatory variable -- the code below allows a matrix of pairwise correlation coefficients to be generated. 

In [ ]:
model_variables = ['total_pop', 'bachelors>', 'veterans',
       'white_collar', 'pink_collar', 'blue_collar', 'median_income',
       'no_health_insurance', 'p_male', 'p_white', 'p_black', 'p_asian',
       'p_latino', 'english_speaking', 'immigrants', 'gen_z', 'gen_y', 'gen_x',
       'baby_boomer', 'silent_gen']
corr_df = df[model_variables]
corr_matrix = corr_df.corr(method="pearson")

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(corr_matrix,center=0,cmap="RdBu",annot=True)

In [ ]:
## VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

corr_df = add_constant(corr_df)
pd.Series([variance_inflation_factor(corr_df.values,i)for i in range(corr_df.shape[1])],index=corr_df.columns)


Remove:
* p_white -p_black
* pink collar - blue collar
* bachelors - correlated with blue collar, + median income, +asian
* baby_boomer, -genz,geny, +silent_gen (old people tend to live together, young people live together)
* silent_gen, gen_y


* Remove: p_white, pink_collar,blue_collar, p_white,baby_boomer



In [ ]:
df["bachelors"] = df["bachelors>"]
model_variables = ['bachelors', 'veterans',
        'blue_collar',
       'no_health_insurance', 'p_male', 'p_white', 'p_asian',
       'p_latino', 'english_speaking', 'immigrants', 'gen_z', 'gen_y', 'gen_x']
corr_df = df[model_variables]
corr_df = add_constant(corr_df)
pd.Series([variance_inflation_factor(corr_df.values,i)for i in range(corr_df.shape[1])],index=corr_df.columns)

Extremely low correlation between variables now

In [ ]:
## Determine Correlation with p_dem_16 and swing
import statsmodels.formula.api as smf
df["bachelors"] = df["bachelors>"]
model_variables = ['bachelors', 'veterans',
        'blue_collar',
       'no_health_insurance', 'p_white', 'p_asian',
       'p_latino', 'english_speaking', 'immigrants', 'gen_z', 'gen_y', 'gen_x']
str_model_variables = ' + '.join(model_variables) #Note that we use :1 to remove Leave from the list, as it is the dependent variable
lm = smf.ols(formula='p_gop_16 ~ '+str_model_variables, data=df).fit()
print(lm.summary())
# adding oldergen only increases r-squared by 2 percent
# Removing both total population and median income results in a 1.7% drop in R-Squared Value

In [ ]:
## Determine Correlation with p_dem_16 and swing
import statsmodels.formula.api as smf
df["bachelors"] = df["bachelors>"]
model_variables = ['bachelors', 'veterans','blue_collar','p_black', 'p_white','p_asian',
        'english_speaking', 'gen_y', 'gen_x']
str_model_variables = ' + '.join(model_variables) #Note that we use :1 to remove Leave from the list, as it is the dependent variable
lm = smf.ols(formula='swing ~ '+str_model_variables, data=df).fit()
print(lm.summary())
# Dem factors: 0.424
#Remove health_insurance, p_latino,immigrants,gen_z
# Addition of Older gen, not substantial
#inclusion of both p_black and p_white, results in a 1% increase

#model_variables = ['bachelors', 'veterans','blue_collar','p_black', 'p_white','p_asian',
#        'english_speaking', 'gen_y', 'gen_x']

Results from the OLS summary indicates that both the factors determining p_dem_16 vote is different from what caused the swing in 2020, with more factors being involved in swing.

In [ ]:
## Determine Correlation with p_dem_16 and swing
import statsmodels.formula.api as smf
df["bachelors"] = df["bachelors>"]
model_variables = ['bachelors', 'veterans','blue_collar','p_black', 'p_white','p_asian',
        'english_speaking', 'gen_y', 'gen_x']
str_model_variables = ' + '.join(model_variables) #Note that we use :1 to remove Leave from the list, as it is the dependent variable
lm = smf.ols(formula='p_gop_20 ~ '+str_model_variables, data=df).fit()
print(lm.summary())
# Using 2016 factors, variables R-Squared = 0.676, explains the 2020 results better than 2016 
# Using Swing Variables: 0.629, worse explainor


## Running Linear Regression Model and Plotting resiudlas on map:
# Normalise Residuals from -1 to 1

In [ ]:
lr = LinearRegression()
def scale(x, out_range=(-1, 1), axis=None):
    domain = np.min(x, axis), np.max(x, axis)
    y = (x - (domain[1] + domain[0]) / 2) / (domain[1] - domain[0])
    return y * (out_range[1] - out_range[0]) + (out_range[1] + out_range[0]) / 2


In [ ]:
model_variables = ['bachelors', 'veterans',
        'blue_collar',
       'no_health_insurance', 'p_white', 'p_asian',
       'p_latino', 'english_speaking', 'immigrants', 'gen_z', 'gen_y', 'gen_x']
X = df[model_variables]
Y = df[["p_gop_16"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_gop_16"] = pred
df["resi_gop_16"] = df["p_gop_16"] - df["pred_gop_16"]
df["resi_gop_16"] = scale(df["resi_gop_16"])
df["resi_gop_16"].describe()
us_boundaries["resi_gop_16"]= df["resi_gop_16"]

In [ ]:
data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

In [ ]:
plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_gop_16:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_gop_16:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)
alt.layer(plot,outline)

* Red means that the model underpredicts the number of dem votes in the area 
* Blue means the model overpredicts the number of dem votes in the area.

In [ ]:
model_variables = ['veterans','p_asian','p_latino','p_white']
X = df[model_variables]
Y = df[["swing"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_swing"] = pred
df["resi_swing"] = df["swing"] - df["pred_swing"]
df["resi_swing"] = scale(df["resi_swing"])
df["resi_swing"].describe()
us_boundaries["resi_swing"]= df["resi_swing"]

data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_swing:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_swing:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)
alt.layer(plot,outline)

## Can maybe ask which was the strongest driver of dems winning/ republicans loosing
* Was it economic,social, demographic factors. 

In [ ]:
## Economic
X = ['veterans','white_collar','pink_collar','blue_collar','median_income','no_health_insurance']
X = df[X]
Y = df[["swing"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_swing_e"] = pred
df["resi_swing_e"] = df["swing"] - df["pred_swing_e"]
df["resi_swing_e"] = scale(df["resi_swing_e"])
df["resi_swing_e"].describe()
us_boundaries["resi_swing_e"]= df["resi_swing_e"]

data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_swing_e:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_swing_e:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)

alt.layer(plot,outline)


In [ ]:
## Demographic
X = ['total_pop','p_male','p_white','p_black','p_asian','p_latino','immigrants','gen_z','gen_y','gen_x','baby_boomer','silent_gen']
X = df[X]
Y = df[["swing"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_swing_d"] = pred
df["resi_swing_d"] = df["swing"] - df["pred_swing_d"]
df["resi_swing_d"] = scale(df["resi_swing_d"])
df["resi_swing_d"].describe()
us_boundaries["resi_swing_d"]= df["resi_swing_d"]

data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_swing_d:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_swing_d:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)

alt.layer(plot,outline)


In [ ]:
## Social
X = ['bachelors','english_speaking']
X = df[X]
Y = df[["swing"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_swing_s"] = pred
df["resi_swing_s"] = df["swing"] - df["pred_swing_s"]
df["resi_swing_s"] = scale(df["resi_swing_s"])
df["resi_swing_s"].describe()
us_boundaries["resi_swing_s"]= df["resi_swing_s"]

data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_swing_s:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_swing_s:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)

alt.layer(plot,outline)


### Comparing 2016 factor weights to 2020 factor weights

In [ ]:
#2016
model_var = ['p_white','pink_collar','bachelors','gen_x','white_collar','veterans','no_health_insurance','english_speaking','p_asian','p_latino','gen_y']
X = df[model_var]
Y = df[["p_gop_16"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_gop_16"] = pred
df["resi_gop_16"] = df["p_gop_16"] - df["pred_gop_16"]
df["resi_gop_16"] = scale(df["resi_gop_16"])
df["resi_gop_16"].describe()
us_boundaries["resi_gop_16"]= df["resi_gop_16"]

data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_gop_16:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_gop_16:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)
coef_16 = dict(zip(model_var,lr.coef_[0]))
print(coef_16)
alt.layer(plot,outline)




In [ ]:
#2020
model_var = ['p_white','pink_collar','bachelors','gen_x','white_collar','veterans','no_health_insurance','english_speaking','p_asian','p_latino','gen_y']
X = df[model_var]
Y = df[["p_gop_20"]]
lr.fit(X,Y)
pred = lr.predict(X)
# Calculate Residuals
df["pred_gop_20"] = pred
df["resi_gop_20"] = df["p_gop_20"] - df["pred_gop_20"]
df["resi_gop_20"] = scale(df["resi_gop_20"])
df["resi_gop_20"].describe()
us_boundaries["resi_gop_20"]= df["resi_gop_20"]

data_geo = alt.InlineData(values = us_boundaries.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

plot = alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).project(type='albersUsa').encode(
    color=alt.Color('properties.resi_gop_20:Q', scale=alt.Scale(scheme='redblue',domain=[-1,1])),
    tooltip=['properties.county_name:N','properties.resi_gop_20:Q','properties.state:N']
).properties(
    width=800,
    height=600
)
outline = alt.Chart(state_geo).mark_geoshape(stroke='black', fillOpacity=0).encode(tooltip=["properties.state_name:N"]).project(
    type='albersUsa'
).properties(
    width=800,
    height=600
)
coef_20 = dict(zip(model_var,lr.coef_[0]))
print(coef_20)
alt.layer(plot,outline)




In [ ]:
resi_under_16 = df[df['resi_gop_16']>-0.5] ## Many counties heavily under predicted
resi_under_20 = df[df['resi_gop_20']>-0.5] ##Less counties under predicted
resi_over_16 = df[df['resi_gop_16']>0.5] ## Many counties heavily under predicted
resi_over_20 = df[df['resi_gop_20']>0.5] ##Less counties under predicted

#What this indicates is that baseline comparison would be lower therefore factor importance for 2016 should be significantly higher ??


print(f'under16: {len(resi_under_16)},under20:{len(resi_under_20)}; over_16:{len(resi_over_16)},over_20:{len(resi_over_20)}')
# With immigrants: 16>-0.5: 3040;20>0.5:117
# Without Immigrants: 16>-0.5: 3044;20>0.5:106
# With baby boomer, silent_gen: 16>-0.5: 3041;20>0.5:52

#Similar numbers of over and under predictions above -0.5 and 0.5
# under16: 3066,under20:3017; over_16:70,over_20:25
# therefore models are similar in predictive capabilities and coefficients can thus be directly compared 

In [ ]:
plt.figure(figsize=(14,5))
sns.lineplot(list(coef_16.keys()),list(coef_16.values()),label='2016',color="r",marker='o')
sns.lineplot(list(coef_20.keys()),list(coef_20.values()),label='2020',color="b",marker='+')
plt.xticks( rotation='90')
plt.legend()

Education, gen_y, veterans,white collar played a larger role in the 2020 election
* Trumps voter base has remained fairly consistent, largely losing followers from veterans,gen_x and a portion of the highly educated.
* Largely shows that trumps voter base isnt driven by his policies but mainly by faith/herd behaviour, many of recent policies including intolerant behaviours should have swayed the minority voter base expecting a decrease however that is not see here.
* 

## Geographically weighted statistics

In [ ]:
import numpy as np

#GWR below requires the "centroids" of each geographical area
centroids = np.array([[c.x,c.y] for c in us_boundaries.geometry.centroid])

#These are all the variables we want to apply geographically-weighted statistics
refined_vars=['p_white','pink_collar','bachelors','gen_x','white_collar','veterans','no_health_insurance','english_speaking','p_asian','p_latino','gen_y']
coeff_names = ['intercept']
map_vars = []
for var in refined_vars:
    coeff_names.append('coeff_'+var)
    map_vars.append('properties.coeff_'+var)

In [ ]:
from mgwr.gwr import GWR, MGWR
us_boundaries["bachelors"] = df["bachelors"]
#Define GWR model
model = GWR(centroids,us_boundaries['p_gop_20'].to_numpy().reshape((-1,1))
            ,us_boundaries[refined_vars].to_numpy(),bw=50,kernel='bisquare',fixed=False)
gw_results = model.fit()

In [ ]:
#GWR or GWRResult does not calculate geographically-weighted correlation coefficients for all variables
#So, this is an adapted version of https://github.com/pysal/mgwr/blob/master/mgwr/gwr.py#L1092
def all_corr(results,variables):
    """
    Computes  local correlation coefficients (n, (((p+1)**2) + (p+1) / 2) within a geographically
    weighted design matrix
    Returns one array with the order and dimensions listed above where n
    is the number of locations used as calibrations points and p is the
    number of explanatory variables; +1 accounts for the dependent variable.
    Local correlation coefficient is not calculated for constant term.
    """
    #print(self.model)
    x = results.X
    y = results.y
    x = np.column_stack((x,y))
    w = results.W
    nvar = x.shape[1]
    nrow = len(w)
    if results.model.constant:
        ncor = (((nvar - 1)**2 + (nvar - 1)) / 2) - (nvar - 1)
        jk = list(combo(range(1, nvar), 2))
    else:
        ncor = (((nvar)**2 + (nvar)) / 2) - nvar
        jk = list(combo(range(nvar), 2))
    corr_mat = np.ndarray((nrow, int(ncor)),dtype=dict)
    
    for i in range(nrow):
        wi = w[i]
        sw = np.sum(wi)
        wi = wi / sw
        tag = 0

        for j, k in jk:
            val = corr(np.cov(x[:, j], x[:, k], aweights=wi))[0][1] 
            corr_mat[i,tag] = {"var": variables[j-1]+"_"+variables[k-1], "var_1": variables[j-1], "var_2": variables[k-1], "value": val}
            tag = tag + 1
            
    return corr_mat

In [ ]:
from mgwr.diagnostics import corr
from itertools import combinations as combo

corr_matrix = all_corr(gw_results,refined_vars+['p_gop_20'])
#Filter only those correlation coefficients against the Leave vote
corr_2 = [{d['var']: d['value'] for d in x if d['var_2'] == 'p_gop_20'} for x in corr_matrix]
corr_coeffs = pd.DataFrame.from_records(corr_2)

In [ ]:
#Check if we have what we wanted (all variables correlated with Leave)
corr_coeffs.head()

In [ ]:
corr_df = pd.merge(us_boundaries,corr_coeffs,left_index=True,right_index=True)
map_vars = ['properties.'+v for v in corr_coeffs.columns.values ]
corr_geo = alt.InlineData(values = corr_df.to_json(),
                       format = alt.DataFormat(property='features',type='json'))

In [ ]:
print(corr_coeffs.columns.values)
alt.Chart(corr_geo 
).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(color=alt.Color(alt.repeat('repeat'), type='quantitative', scale=alt.Scale(scheme='purplegreen')),
    tooltip=['properties.state_name:N',alt.Tooltip(alt.repeat("repeat"), type="quantitative")],
).properties(
    projection={'type': 'identity','reflectY': True},
    width=175,
    height=260,
).repeat(map_vars,columns=4)

In [ ]:

us_boundaries["bachelors"] = df["bachelors"]
#Define GWR model
model_16 = GWR(centroids,us_boundaries['p_gop_16'].to_numpy().reshape((-1,1))
            ,us_boundaries[refined_vars].to_numpy(),bw=50,kernel='bisquare',fixed=False)
gw_results = model_16.fit()

In [ ]:
corr_matrix = all_corr(gw_results,refined_vars+['p_gop_16'])
#Filter only those correlation coefficients against the Leave vote
corr_2 = [{d['var']: d['value'] for d in x if d['var_2'] == 'p_gop_16'} for x in corr_matrix]
corr_coeffs = pd.DataFrame.from_records(corr_2)

In [ ]:
corr_coeffs.head()

In [ ]:
corr_df = pd.merge(us_boundaries,corr_coeffs,left_index=True,right_index=True)
map_vars = ['properties.'+v for v in corr_coeffs.columns.values ]
corr_geo = alt.InlineData(values = corr_df.to_json(),
                       format = alt.DataFormat(property='features',type='json'))

In [ ]:
print(corr_coeffs.columns.values)
alt.Chart(corr_geo 
).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(color=alt.Color(alt.repeat('repeat'), type='quantitative', scale=alt.Scale(scheme='purplegreen')),
    tooltip=['properties.state_name:N',alt.Tooltip(alt.repeat("repeat"), type="quantitative")],
).properties(
    projection={'type': 'identity','reflectY': True},
    width=175,
    height=260,
).repeat(map_vars,columns=4)

> **<font color='red'>GROUP DISCUSSION POINT 5</font>**
>
> The maps are correlation coefficients for the variables in the order listed above with %leave.
>
> * Study the maps. Which variables are more regionally distinct?
> * Can you offer explanations as to why this might be?


Scanning across the maps and making systematic claims about combinations of relationships is challenging. Clustering LAs on their gw-correlation coefficients might help. In the code below, each LA is summarised according to its geographically-weighted correlation coefficient and agglomerative hierarchical cluster analysis (HCA) is used to identify groups of LAs that share *similar combinations of relationship*. LAs are then ‘agglomerated’ into groups iteratively by merging the most similar LAs. This continues until all LAs are merged into a single group. We can evaluate the clustering visually by plotting a dendrogram depicting this agglomeration process, and numerically by considering Average Silhouette Width (ASW) values, calculated at different cuts (number of clusters) of the dendrogram. 

We won't go into length about the choice of cluster analysis: if two variables are included that represent the same concept, then that concept is given undue weight. Variables were carefully selected by visually inspecting correlation matrices of the geographically-weighted correlation coefficients – similar to the approach for assessing collinearity in regression. The input variables selected via this process are: __Christian, degree-educated, no car, not good health, white.__



#### Clustering 

* Variables selected for use: p_asian,english_speaking, bachelors>,veterans,no_health_insurance
* P_asian has similar patterns to white_collar, pink_collar, gen_y
* p_latino has weak correlation with gop votes in general
* p_white similar to english_speaking

In [ ]:
from sklearn.preprocessing import StandardScaler

#First we standardise the variables
cluster_variables = ['p_asian_p_gop_20','english_speaking_p_gop_20','bachelors_p_gop_20','veterans_p_gop_20','no_health_insurance_p_gop_20' ]
scaler = StandardScaler()
scaled_coefficients=scaler.fit_transform(corr_coeffs[cluster_variables])
scaled_df = pd.DataFrame(scaled_coefficients,columns=corr_coeffs[cluster_variables].columns)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score,silhouette_samples

cluster_results = [None]*12
for i in range(2,12): 
    ac =  AgglomerativeClustering(linkage='ward', n_clusters=i)
    cr = ac.fit(scaled_df.values)
    cluster_results[i] = cr
    print(str(i)+" clusters. Avg silhouette score:",silhouette_score(scaled_df.values,cr.labels_))
    
#With p_asian, best cluster 2 clusters. Avg silhouette score: 0.22718602619530998
#Replace asian with white_collar: 2Cluster: worse 0.17893
#gen_y 2 clusters. Avg silhouette score: 0.18899976772592417
#including both gen_y and asian = 2 clusters. Avg silhouette score: 0.18985285130638524

In [ ]:
hc_result =  cluster_results[2] 
corr_df['cluster_membership'] = hc_result.labels_
corr_coeffs['cluster_membership'] = hc_result.labels_
corr_coeffs

In [ ]:
sil_scores = pd.DataFrame(
    np.column_stack((silhouette_samples(scaled_df.values,hc_result.labels_),hc_result.labels_))
    ,columns=['silhouette','cluster_m']).reset_index()

In [ ]:
alt.Chart(sil_scores).mark_bar(
        ).encode(
            x='silhouette:Q',
            y=alt.Y('index:O',sort=alt.SortField(field="silhouette", order='descending'),axis=None),
            color=alt.Color('cluster_m:N',scale=alt.Scale(scheme='accent')),
            facet='cluster_m:N'
        ).properties(width=150,height=250).resolve_scale(y='independent')

In [ ]:
alt.Chart(alt.InlineData(values = corr_df.to_json(),
                       format = alt.DataFormat(property='features',type='json')) 
         ).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
        ).encode(
            color=alt.Color('properties.cluster_membership:N',scale=alt.Scale(scheme='accent')),
            tooltip=['properties.county_name:N','properties.cluster_membership:N']
        ).properties(
            projection={'type': 'identity','reflectY': True},
            width=800,
            height=600,
        )